## Populate an RDF database

This notebook reports the main steps to download CSV files, process them and create an RDF dataset from them accordingly to an ontology. 

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

In [1]:
# required libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path

# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
stats1920Url = path + '/inDepthSoccerStats/2019-2020.csv'
stats1819Url = path + '/inDepthSoccerStats/2018-2019.csv'
playersUrl = path + '/inDepthSoccerStats/players.csv'
appUrl = path + '/inDepthSoccerStats/appearances.csv'

# country codes
# countriesURL = path + '/data/countryCodes/wikipedia-iso-country-codes.csv'

# saving folder
savePath =  path + '/rdf/'

## Soccer Stats

In [2]:
# Load the CSV files in memory
stats1920 = pd.read_csv(stats1920Url, sep=',', index_col='player_name')
stats1819 = pd.read_csv(stats1819Url, sep=',', index_col='player_name')
players = pd.read_csv(playersUrl, sep=',', index_col='player_id')
app = pd.read_csv(appUrl, sep=',', index_col='appearance_id')
# cast year to int. If type(year) = str --> Literal= year-01-01
#stats1920.astype({'year': 'int32'}).dtypes
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
#countries = pd.read_csv(countriesURL, sep=',', index_col='Name', keep_default_na=False, na_values=['_'])

#stats1920.info()
#players.info()

We need to install <code>RDFLib</code>

<code>pip3 install rdflib </code> [Documentation](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)

In [3]:
# Load the required libraries
#!pip install rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace, term
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD



In [4]:
# Construct the country and the movie ontology namespaces not known by RDFlib
#CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
DCSSO = Namespace("http://www.dei.unipd.it/db2/dcsso#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
#g.bind("countries", CNS)
g.bind("dcsso", DCSSO)

#term.bind(
#    XSD.double,
#    float,
#   constructor=float,
#    lexicalizer=lambda val: f"{val:f}",
#    datatype_specific=True
#)

In [5]:
import datetime
#!pip install unidecode
#!pip install googlesearch-python
from unidecode import unidecode
from itertools import permutations
from difflib import SequenceMatcher
from googlesearch import search

#string parsing

def nameToRef(name):
    return unidecode(name.replace(" ",""))

def hyphenize(s):
    return unidecode(s.lower().replace(" ","-").replace("&#039;","'"))

def cleanChars(item):
    return item.str.replace("ć", "c").str.replace("ğ","g").str.replace("İ","i").str.replace("-scaron-","s")

#google lookup for wikidata item
def checkEqualsWD(c1, c2):
    URL1 = next(search(c1.replace("-", " ") + " football wikidata", num_results=1))
    URL2 = next(search(c2.replace("-", " ") + " football wikidata", num_results=1))
    if(URL1 == URL2):
        return True
    else:
        return False

#permutations strategy

def genSeqByLength(perm, length):
    newName = ""
    for j in range(0, length - 1):
        newName = newName + perm[j] + "-"
    newName = newName + perm[length - 1]
    return newName

def match_seq(splitS1, list):
    resultList = []
    #iterate on all possible permutations
    length = -1
    if(len(list) == 1):
        length = len(list[0].split("-"))
        uniqueItem = list[0]
    for perm in permutations(splitS1):
        if(length != -1):
            newName = genSeqByLength(perm, length)
            if(uniqueItem == newName):
                return [uniqueItem]
        #consider all lengths from 2 to n
        else:
            for i in range(2, len(splitS1) + 1):
                newName = genSeqByLength(perm, i)
                for item in list:
                    if(item == newName):
                        resultList.append(newName)
    return resultList

#multiple matches resolution

def getAppsByID(ID, apps):
    return np.size(apps[apps['player_id'] == ID], 0);

#I: number of games in the season, candidate players, list describing single appearances
#O: player from players file, or empty Series
def solve_with_apps(statsGames, somePlayers, appsCol):
    for ind in somePlayers.index:
        if(statsGames == getAppsByID(ind, appsCol)):
            player = somePlayers[somePlayers.index == ind].iloc[0]
            return player
    return pd.Series([])

def solve_with_apps_approx(statsGames, somePlayers, appsCol):
    minDiff = 50
    minInd = 0
    for ind in somePlayers.index:
        if(abs(statsGames - getAppsByID(ind, appsCol)) < minDiff):
            minDiff = abs(statsGames - getAppsByID(ind, appsCol))
            minInd = ind
    if(minDiff <= 5):
        player = somePlayers[somePlayers.index == minInd].iloc[0] 
        return player
    return pd.Series([])

In [7]:
%%time

#select only months and years from appearance dates
appMonth = app['date'].str.split("/").str[1]
appYear = app['date'].str.split("/").str[2]
#select appearances from 18/19 season
is1819 = ((appYear == "2018") & (appMonth >= "08")) | ((appYear == "2019") & (appMonth <= "06"))
app1819 = app[is1819]
playerCodes = cleanChars(players['player_code'])

#iterate on stats file
exact_matches = no_matches = resolved_google = resolved_permS = resolved_permP = resolved_max_sim = unable_to_solve = 0
for index, row in stats1819.iterrows():
    mode = "NONE"
    player = pd.Series([])
    statsName = hyphenize(index).replace("'","")
    
    matchedPlayers = players[playerCodes == statsName]
    
    #multiple rows with same name in stats mapped to a single player are ok (he has changed team during the season)
    if(np.size(matchedPlayers, 0) > 0):
        mode = "MATCH"
       
    if(mode == "NONE"):
        #split name in stats and use permutations strategy
        splitStatsName = statsName.split("-")
        if(len(splitStatsName) >= 2):
            matchedCodes = match_seq(splitStatsName, playerCodes)
            if(len(matchedCodes) > 0):
                matchedPlayers = players[players['player_code'].isin(matchedCodes)]
                mode = "PERM1"

    if(mode == "NONE"):
        maxSim = 0
        maxC = ""
        for c in playerCodes:
            sm = SequenceMatcher(None, statsName, c)
            #do not proceed if the upper bound is too small
            if(sm.real_quick_ratio() >= 0.5):
                #remember: similarity is not commutative
                simm = sm.ratio()
                if(simm > maxSim):
                    maxSim = simm
                    maxC = c
                    
                #if sim is big enough, try permutation strategy with name from players file
                if(simm >= 0.6):
                    splitC = c.split("-")
                    if(len(splitC) >= len(splitStatsName) and len(splitStatsName) >= 2):
                        matchedCodes = match_seq(splitC, [statsName])
                        if(len(matchedCodes) > 0):
                            newMatchedPlayers = players[players['player_code'] == c]
                            matchedPlayers = pd.concat([matchedPlayers, newMatchedPlayers])

        if(maxSim >= 0.95):
            matchedPlayers = players[playerCodes == maxC]
            mode = "MAXSIM"
        elif(np.size(matchedPlayers, 0) > 0):
            mode = "PERM2"
        elif(checkEqualsWD(statsName, maxC) == True):
            matchedPlayers = players[playerCodes == maxC]
            mode = "GOOGLE"

    #managing results of any method
    matches = np.size(matchedPlayers, 0)
    if(matches == 1):
        player = matchedPlayers.iloc[0]
    #more than 1 possible match
    if(matches > 1):
        player = solve_with_apps_approx(row['games'], matchedPlayers, app1819)
        if(len(player) == 0):
            mode = "NONE"
            unable_to_solve += 1
    
    if(mode == "NONE"):
        no_matches += 1
    elif(mode == "PERM1"):
        resolved_permS += 1
    elif(mode == "PERM2"):
        resolved_permP += 1
    elif(mode == "GOOGLE"):
        resolved_google += 1
    elif(mode == "MATCH"):
        exact_matches += 1
    elif(mode == "MAXSIM"):
        resolved_max_sim += 1

    if(mode == "NONE"):
        print("NONE: "+statsName+", matches: "+str(matches))
    elif(mode != "MATCH"):
        print(mode+": "+statsName+" --> "+player['player_code'])

PERM1: son-heung-min --> heung-min-son
PERM2: bobby-reid --> bobby-de-cordova-reid
GOOGLE: jose-holebas --> jose-cholevas
NONE: sokratis, matches: 0
NONE: jonny, matches: 0
PERM1: ki-sung-yueng --> sung-yueng-ki
GOOGLE: joseph-gomez --> joe-gomez
GOOGLE: mat-ryan --> mathew-ryan
NONE: kepa, matches: 0
NONE: bruno, matches: 2
NONE: franck-zambo, matches: 0
NONE: roger, matches: 3
GOOGLE: morales --> lucas-morales
NONE: ezequiel-avila, matches: 0
GOOGLE: daniel-parejo --> dani-parejo
NONE: angel, matches: 0
GOOGLE: muniain --> iker-muniain
GOOGLE: santiago-cazorla --> santi-cazorla
GOOGLE: etxeita --> xabi-etxeita
PERM1: juan-camilo-hernandez --> juan-hernandez
NONE: alfonso, matches: 0
PERM1: wu-lei --> lei-wu
NONE: keko, matches: 0
NONE: yuri, matches: 0
NONE: aduriz, matches: 0
MAXSIM: cristian-rivera --> christian-rivera
NONE: nyom, matches: 0
NONE: bernardo, matches: 3
NONE: papakouly-diop, matches: 0
NONE: tono, matches: 2
NONE: zaldua, matches: 0
NONE: mario, matches: 0
NONE: robe

In [8]:
#print statistics
statsRows = np.size(stats1819, 0);
print("   --- STATISTICS ---")
tot_matches = exact_matches + + resolved_google + resolved_permS + resolved_permP + resolved_max_sim
print("Total matches:                            "+"{:5d}".format(tot_matches)+" -- percentage: " + "{:.2f}%".format(tot_matches*100/statsRows))
print("  ---> exact matches:                     "+"{:5d}".format(exact_matches)+" -- percentage: " + "{:.2f}%".format(exact_matches*100/statsRows)) 
print("  ---> resolved permutating statsName:    "+"{:5d}".format(resolved_permS)+" -- percentage: " + "{:.2f}%".format(resolved_permS*100/statsRows))
print("  ---> resolved permutating player code:  "+"{:5d}".format(resolved_permP)+" -- percentage: " + "{:.2f}%".format(resolved_permP*100/statsRows))
print("  ---> resolved with google:              "+"{:5d}".format(resolved_google)+" -- percentage: " + "{:.2f}%".format(resolved_google*100/statsRows))
print("  ---> resolved with max sim.:            "+"{:5d}".format(resolved_max_sim)+" -- percentage: " + "{:.2f}%".format(resolved_max_sim*100/statsRows))
print("No matches:                               "+"{:5d}".format(no_matches)+" -- percentage: " + "{:.2f}%".format(no_matches*100/statsRows))
print("  ---> unable to resolve between matches: "+"{:5d}".format(unable_to_solve)+" -- percentage: " + "{:.2f}%".format(unable_to_solve*100/statsRows))
print("  ---> zero matches found:                "+"{:5d}".format(no_matches - unable_to_solve)+" -- percentage: " + "{:.2f}%".format((no_matches - unable_to_solve)*100/statsRows))

   --- STATISTICS ---
Total matches:                             2515 -- percentage: 96.92%
  ---> exact matches:                      2432 -- percentage: 93.72%
  ---> resolved permutating statsName:       31 -- percentage: 1.19%
  ---> resolved permutating player code:     12 -- percentage: 0.46%
  ---> resolved with google:                 28 -- percentage: 1.08%
  ---> resolved with max sim.:               12 -- percentage: 0.46%
No matches:                                  80 -- percentage: 3.08%
  ---> unable to resolve between matches:    14 -- percentage: 0.54%
  ---> zero matches found:                   66 -- percentage: 2.54%


In [ ]:
%%time 
# measure execution time

# just a variable to diversify between footballers with the same name
a = 0
# iterate over the movies dataframe
for index, row in stats1920.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the footballer name as URI
    name = unidecode(index.replace("&#039;","'"))
    ref = name.replace(" ","")
    # add a unique code if there are players with the same name
    if(np.size(stats1920[stats1920.index == name], 0) > 1):
        #print(name + " : " + str(np.size(stats1920[stats1920.index == name], 0)))
        a = a + 1
        ref = ref + str(a)
    
    Footballer = URIRef(DCSSO[ref])
    # Add triples using store's add() method.
    g.add((Footballer, RDF.type, DCSSO.Footballer))
    g.add((Footballer, DCSSO['name'], Literal(name, datatype=XSD.string)))
    #g.add((Movie, MO['releaseYear'], Literal(row['year'], datatype=XSD.gYear)))
    
    leagueName = unidecode(row['league'])
    League = URIRef(DCSSO[leagueName.replace(" ","")])
    g.add((League, DCSSO['name'], Literal(row['league'], datatype=XSD.string)))
    
    teams = unidecode(row['teams_played_for'].replace("&#039;","'").strip())
    
    membRef = ref+"_1920_"+teams.replace(",","_").replace(" ","")
    Memb = URIRef(DCSSO[membRef])
    g.add((Footballer, DCSSO['hasMembership'], Memb))
    
    for teamName in teams.split(','):
        teamRef = teamName.replace(" ","")
        Team = URIRef(DCSSO[teamRef])
        g.add((Team, DCSSO['name'], Literal(teamName, datatype=XSD.string)))
        g.add((Team, DCSSO['participatesIn'], League))
        g.add((Memb, DCSSO['forTeam'], Team))
    
    #statistics
    g.add((Memb, DCSSO['games'], Literal(row['games'], datatype=XSD.int)))
    g.add((Memb, DCSSO['minutes'], Literal(row['minutes_played'], datatype=XSD.int)))
    g.add((Memb, DCSSO['goals'], Literal(row['goals'], datatype=XSD.int)))
    g.add((Memb, DCSSO['npg'], Literal(row['npg'], datatype=XSD.int)))
    g.add((Memb, DCSSO['assists'], Literal(row['assists'], datatype=XSD.int)))
    g.add((Memb, DCSSO['keyPasses'], Literal(row['key_passes'], datatype=XSD.int)))
    g.add((Memb, DCSSO['shots'], Literal(row['shots'], datatype=XSD.int)))
    g.add((Memb, DCSSO['yellowCards'], Literal(row['yellow_cards'], datatype=XSD.int)))
    g.add((Memb, DCSSO['redCards'], Literal(row['red_cards'], datatype=XSD.int)))
    
    g.add((Memb, DCSSO['xG'], Literal(row['xG'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xA'], Literal(row['xA'], datatype=XSD.double)))
    g.add((Memb, DCSSO['npxG'], Literal(row['npxG'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xG90'], Literal(row['xG90'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xA90'], Literal(row['xA90'], datatype=XSD.double)))
    g.add((Memb, DCSSO['npxG90'], Literal(row['npxG90'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xGBuildup'], Literal(row['xGBuildup'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xGChain'], Literal(row['xGChain'], datatype=XSD.double)))
    
        
    #there can be more than one role per footballer
    for rN in row['position'].split(' '):
        g.add((Footballer, DCSSO['role'], Literal(rN.strip(), datatype=XSD.string)))    

#print("Duplicates: "+str(a/2))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'stats1920.rdf', 'w') as file:
    file.write(g.serialize(format='xml'))
    #.decode("utf-8")


# Referential integrity
Note that in RDF we are in an open world situation. We cannot guarantee the referential integrity between the entities. 

## Person

Let us generate the RDF data relative to the movie workers.

In [ ]:
# Load the CSV files in memory
people = pd.read_csv(namesUrl, sep=',', index_col='imdb_name_id', keep_default_na=False, na_values=['_'])

In [ ]:
people.info()

People are modeled with the FOAF ontology. 
Refer to [FOAF Documentation](http://xmlns.com/foaf/spec/)

In [ ]:
#create a new graph
g = Graph()

In [ ]:
%%time 
#measure execution time

#iterate over the person dataframe
for index, row in people.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the person id as URI
    Person = URIRef(MO[index])
    g.add((Person, RDF.type, FOAF.Person))
    # Add triples using store's add() method.
    g.add((Person, FOAF['name'], Literal(row['name'], datatype=XSD.string)))
    if row['date_of_birth'] != '':
        try:
            datetime.datetime.strptime(str(row['date_of_birth']), '%Y-%m-%d')
            g.add((Person, MO['birthday'], Literal(row['date_of_birth'], datatype=XSD.date)))
        except ValueError:
            # probably it's the year alone
            # check length
            if (len(row['date_of_birth'])==4):
                #it is a year
                g.add((Person, MO['birthday'], Literal(row['date_of_birth']+"-01-01", datatype=XSD.date)))
    
    if row['place_of_birth'] != '':
        g.add((Person, MO['birthplace'], Literal(row['place_of_birth'], datatype=XSD.string)))
    
    # check if the death day is not empty--i.e., the person is still alive
    if row['date_of_death'] != '':
        try:
            datetime.datetime.strptime(str(row['date_of_death']), '%Y-%m-%d')
            g.add((Person, MO['deathDay'], Literal(row['date_of_death'], datatype=XSD.date)))
        except ValueError:
            # probably it's the year alone
            # check length
            if (len(row['date_of_death'])==4):
                #it is a year
                g.add((Person, MO['deathDay'], Literal(row['date_of_death']+"-01-01", datatype=XSD.date)))
        
# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("mo", MO)

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'names.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))

## Person-Movie Join

In [ ]:
# Load the CSV files in memory
join = pd.read_csv(joinTableUrl, sep=',', index_col='imdb_title_id', keep_default_na=False, na_values=['_'])

In [ ]:
#create a new graph
g = Graph()

In [ ]:
#regular expressions
import re
actor = re.compile('act*')

In [ ]:
%%time 
#measure execution time

#iterate over the join table dataframe
for index, row in join.iterrows():
    # Create the node about the movie
    # note that we do not add this resource to the database (created before)
    Movie = URIRef(MO[index])
    
    # Create the node about the person
    # note that we do not add this resource to the database (created before)
    Person = URIRef(MO[row['imdb_name_id']])
    # get the role of the person
    role = row['category']
    
    # we have an actor or actress
    if actor.match(role): 
        g.add((Person, MO['acted'], Movie))
    elif (role=='director'):
        g.add((Person, MO['directed'], Movie))
    else:
        # note that, with the defined ontology, we cannot caracterize the specific role of this person in the movie. 
        # why?
        g.add((Person, MO['worked'], Movie))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("mo", MO)

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'name_movie_join.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))

## Awards - Oscars data
Note that if we do not check the referential integrity then we could produce ghost triple movie-nominee-oscar where the movie is not in the RDF graph.

On the other hand, we can check if an actor or a movie exists by using the DataFrame in Python. Note that this is an external check and not a constraints met by the RDF DB.



In [ ]:
# Load the CSV files in memory
oscars = pd.read_csv(oscarsUrl, sep=',', keep_default_na=False, na_values=['_'])

In [ ]:
from num2words import num2words
import string
import re
#create a new graph
g = Graph()

In [ ]:
%%time
#iterate over the join table dataframe
for index,row in oscars.iterrows():
    #create the oscar with a custom id 
    cat = re.sub(r'[^\w\s]','',row['category'])
    Oscar = URIRef(MO['oscar_'+cat.replace(" ", "").lower()+'_'+ str(num2words(row['ceremony'], to='ordinal'))])
    
    # check if there already is at least a triple about this oscar
    if not (Oscar, None, None) in g:    
        # check if the oscar is already in the graph
        g.add((Oscar, RDF.type, MO.Oscar))
        g.add((Oscar, MO['category'], Literal(row['category'].lower(), datatype=XSD.string)))
        g.add((Oscar, MO['year'], Literal(row['year_ceremony'], datatype=XSD.gYear)))
    
    # check if there is a name matching the people, meaning that the oscar can be associated to a person
    if (people["name"] == row["name"]).any() == True :
        #there is a person with this name
        # Create the node about the person
        # note that we do not add this resource to the database (created before)
        Person = URIRef(MO[people[people["name"]==row["name"]].index[0]])
        if row['winner']:
            g.add((Person, MO['winner'], Oscar))
        else:
            g.add((Person, MO['nominated'], Oscar))
    
    # an oscar for a person is also to be considered an oscar for the movie
    # check if the movie is in our DB
    if (movies["original_title"] == row["film"]).any():
        # there is a movie with this title
        Movie = URIRef(MO[movies[movies["original_title"]==row["film"]].index[0]])
        if row['winner']:
            g.add((Movie, MO['winner'], Oscar))
        else:
            g.add((Movie, MO['nominated'], Oscar))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("mo", MO)

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'oscars.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))